In [ ]:
"""
Improving Forecasting by Exogenous Variables

Python Code for SARIMAX Model

"""

#%% Load Packages
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

#%% Function for log transformation
def log_transf(x):
  if x != 0:
    return np.log(x)
  if x == 0:
    return 0

#%% Function to get results for each moving window
def sarimax_window(col_core, features_list, h, window):
  n_train = 222
  n_test = 154
  n_iter = n_test-h+1

  df_US = pd.read_csv('CLEANED_35_Updated.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  df_US = df_US.drop(columns=['SEQNUM','date','Unnamed: 18','Unnamed: 19'])
  df_sarimax_results = pd.DataFrame()
  df_sarimax14 = df_US[[col_core]][h:]
  df_exog14 = df_US[features_list][0:-h]
  df_exog14.index = df_sarimax14.index
  df_sarimax14[features_list] = df_exog14[features_list]
  df_sarimax_old14 = df_sarimax14.loc[df_US[44:].index]

  sarimax_old14_row = []
  df_current_old14 = df_sarimax_old14[int(window-1):int(window-1+n_train+h)]
  df_log_old14 = pd.DataFrame()
  for col in df_current_old14.columns:
    df_log_old14[col] = df_current_old14[col].apply(lambda x: log_transf(x))
  training_old14, testing_old14 = df_log_old14[0:n_train], df_log_old14[n_train:]
  n_test_current = len(testing_old14)

  exog_train_old14 = training_old14[features_list]
  exog_test_old14 = testing_old14[features_list]
  model = SARIMAX(training_old14[col_core], exog=exog_train_old14, order=(4,1,4), seasonal_order=(3,1,1,7), enforce_stationarity=False, enforce_invertibility=False)
  model_fit = model.fit(disp=False)
  forecast = model_fit.predict(len(training_old14), len(training_old14)+(n_test_current-1), exog = exog_test_old14)
  forecast = np.exp(forecast)
  sarimax_old14_row.append(forecast)

  df_sarimax_old14_results = pd.DataFrame(pd.concat(sarimax_old14_row), columns=[col_core])
  df_sarimax_results["h=" + str(h)] = df_sarimax_old14_results[col_core]
  return df_sarimax_results

#%% Function to get forecasts for each horizon
def output_sarimax_single(col_core, features_list, h):
  n_train = 222
  n_test = 154
  n_iter = n_test-h+1

  df_results = []

  if h==1:
    for i in range(int(n_iter)):
      window = i + 1
      df_results.append(sarimax_window(col_core, features_list, h, window))

  if h>1:
    for i in range(int(h-1)):
      window = i + 1
      current_results = sarimax_window(col_core, features_list, h, window)
      df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())
    for i in range(int(n_iter)):
      window = i + 1
      current_results = sarimax_window(col_core, features_list, h, window)
      df_results.append(pd.DataFrame(current_results.iloc[int(h-1)]).transpose())

  df_results_python = pd.concat(df_results,axis=0)
  return df_results_python

#%% Function to combine the forecasts from different horizon
def forecast_h(col_core, features_list):
  df_final_forecast = pd.DataFrame()
  for h in range(1,15):
    df_current = output_sarimax_single(col_core, features_list, h)
    df_final_forecast = pd.concat([df_final_forecast, df_current], axis=1)
  return df_final_forecast

#%% Function to calculate smape values
def smape(y_true, y_pred):
  m = len(y_true)
  n = 0
  sum = 0.0
  e = y_true - y_pred
  for i in range(0,m):
      denom = np.abs(y_true[i]) + np.abs(y_pred[i])
      if (denom != 0.0):
          sum = sum + np.abs(e[i]) / denom
          n = n + 1
  smape = 200 * sum / n
  return smape